In [1]:
# -*- coding: utf-8 -*-
import os, sys
import glob
import json
sys.path.append("..")
sys.path.append("../..")
import dataset_utils.utils as utils
import dataset_utils.config as config
from utils.FaceAligner import FaceAligner

In [7]:
src_dir = os.path.join(config.DS_DIALOG_PILOT, "transformed")
tgt_dir = os.path.join(config.DS_DIALOG_PILOT, "face")
# movie_files = ["omni_dialog_pilot_01.MP4"]
movie_files = ["omni_dialog_pilot_02_01.MP4", "omni_dialog_pilot_02_02.MP4"]
json_dir = os.path.join(config.DS_DIALOG_PILOT, "face_bb_and_landmarks")

In [ ]:
for movie_file in movie_files:
    movie_id = movie_file.split(".")[0]
    src_sub_dir = os.path.join(src_dir, movie_id)
    tgt_sub_dir = os.path.join(tgt_dir, movie_id)
    if not os.path.exists(tgt_sub_dir):
        os.makedirs(tgt_sub_dir)
    if not os.path.exists(json_dir):
        os.makedirs(json_dir)
    json_path = os.path.join(json_dir, "%s.json" % movie_id)
        
    aligner = FaceAligner()

    src_paths = sorted(glob.glob(os.path.join(src_sub_dir, "*.jpg")))
    results = {}
    for i, path in enumerate(src_paths):
        print("%d / %d" % (i+1, len(src_paths)))

        fname = os.path.basename(path)
        img = utils.imread(path)
        bb = aligner.bb(img=img)

        if bb is not None:
            landmarks = aligner.landmarks(img=img, bb=bb)
            # save face img
            face_img = img[bb.top():bb.bottom(), bb.left():bb.right()]
            utils.imwrite(os.path.join(tgt_sub_dir, fname), face_img)
            results[fname] = {}
            results[fname]['bb'] = [bb.left(), bb.top(), bb.right(), bb.bottom()]
            results[fname]["landmarks"] = landmarks

    with open(json_path, "w") as fr:
        json.dump(results, fr, sort_keys=True)
utils.notify("@nogaken detect_face done!!")

1 / 5238
2 / 5238
3 / 5238
4 / 5238
5 / 5238
6 / 5238
7 / 5238
8 / 5238
9 / 5238
10 / 5238
11 / 5238
12 / 5238
13 / 5238
14 / 5238
15 / 5238
16 / 5238
17 / 5238
18 / 5238
19 / 5238
20 / 5238
21 / 5238
22 / 5238
23 / 5238
24 / 5238
25 / 5238
26 / 5238
27 / 5238
28 / 5238
29 / 5238
30 / 5238
31 / 5238
32 / 5238
33 / 5238
34 / 5238
35 / 5238
36 / 5238
37 / 5238
38 / 5238
39 / 5238
40 / 5238
41 / 5238
42 / 5238
43 / 5238
44 / 5238
45 / 5238
46 / 5238
47 / 5238
48 / 5238
49 / 5238
50 / 5238
51 / 5238
52 / 5238
53 / 5238
54 / 5238
55 / 5238
56 / 5238
57 / 5238
58 / 5238
59 / 5238
60 / 5238
61 / 5238
62 / 5238
63 / 5238
64 / 5238
65 / 5238
66 / 5238
67 / 5238
68 / 5238
69 / 5238
70 / 5238
71 / 5238
72 / 5238
73 / 5238
74 / 5238
75 / 5238
76 / 5238
77 / 5238
78 / 5238
79 / 5238
80 / 5238
81 / 5238
82 / 5238
83 / 5238
84 / 5238
85 / 5238
86 / 5238
87 / 5238
88 / 5238
89 / 5238
90 / 5238
91 / 5238
92 / 5238
93 / 5238
94 / 5238
95 / 5238
96 / 5238
97 / 5238
98 / 5238
99 / 5238
100 / 5238
101 / 52

756 / 5238
757 / 5238
758 / 5238
759 / 5238
760 / 5238
761 / 5238
762 / 5238
763 / 5238
764 / 5238
765 / 5238
766 / 5238
767 / 5238
768 / 5238
769 / 5238
770 / 5238
771 / 5238
772 / 5238
773 / 5238
774 / 5238
775 / 5238
776 / 5238
777 / 5238
778 / 5238
779 / 5238
780 / 5238
781 / 5238
782 / 5238
783 / 5238
784 / 5238
785 / 5238
786 / 5238
787 / 5238
788 / 5238
789 / 5238
790 / 5238
791 / 5238
792 / 5238
793 / 5238
794 / 5238
795 / 5238
796 / 5238
797 / 5238
798 / 5238
799 / 5238
800 / 5238
801 / 5238
802 / 5238
803 / 5238
804 / 5238
805 / 5238
806 / 5238
807 / 5238
808 / 5238
809 / 5238
810 / 5238
811 / 5238
812 / 5238
813 / 5238
814 / 5238
